# Анализ изображений и видео

Теперь вам предстоит самостоятельно попробовать распознать людей на изображениях, с помощью моделей компьютерного зрения и библиотек машинного обучения.

In [1]:
import warnings
warnings.filterwarnings('ignore')

!git clone https://github.com/anushkadhiman/YOLOv3-TensorFlow-2.x.git
%cd YOLOv3-TensorFlow-2.x
!pip install -r ./requirements.txt
!wget -P model_data https://pjreddie.com/media/files/yolov3.weights

fatal: destination path 'YOLOv3-TensorFlow-2.x' already exists and is not an empty directory.
/content/YOLOv3-TensorFlow-2.x
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached wget-3.2.zip (10 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.1 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==2.3.1
--2023-04-21 15:35:13--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connect

1. Скачайте исходные данные

In [2]:
from urllib.request import urlretrieve
import os

DATA = ['https://de.ifmo.ru/--openedu/dataprocessing/appliedai/img/20.jpg',
        'https://de.ifmo.ru/--openedu/dataprocessing/appliedai/img/24.jpg',
        'https://de.ifmo.ru/--openedu/dataprocessing/appliedai/img/28.jpg',
        'https://de.ifmo.ru/--openedu/dataprocessing/appliedai/img/23.jpg',
        'https://de.ifmo.ru/--openedu/dataprocessing/appliedai/img/0.jpg',
        'https://de.ifmo.ru/--openedu/dataprocessing/appliedai/img/26.jpg',
        'https://de.ifmo.ru/--openedu/dataprocessing/appliedai/img/8.jpg',
        'https://de.ifmo.ru/--openedu/dataprocessing/appliedai/img/34.jpg',
        'https://de.ifmo.ru/--openedu/dataprocessing/appliedai/img/12.jpg',
        'https://de.ifmo.ru/--openedu/dataprocessing/appliedai/img/31.jpg']
SCORE_THRESHOLD = 0.5

os.makedirs('data')
images = [f'{i}.jpg' for i in range(1, len(DATA)+1)]
for image, link in zip(images, DATA):
  urlretrieve(link, f'data/{image}')
os.makedirs('output')

2. Выполните распознавание

In [3]:
from yolov3.utils import Load_Yolo_model, detect_image
from IPython.display import display, Image

model = Load_Yolo_model()
for image in images:
  detect_image(model, f'data/{image}', f'output/{image}', score_threshold=SCORE_THRESHOLD)
  display(Image(filename=f'output/{image}', width=600) )

1/1 [==============================] - 1s 1s/step


3. Составьте матрицы ошибок и вычислите метрики
   Правила заполнения:
   - TP (True Positive) — объект классифицирован правильно (т.е. мнение классификатора и эксперта совпало);
   - TN (True Negative) — так как объектов много (не только людей), считать этот параметр всегда равным 0;
   - FP (False Positive) — классификатор говорит, что выделенный объект является человеком, эксперт говорит, что нет;
   - FN (False Negative) — эксперт видит человека, классификатор - нет.

   *Пример:* На фотографии 4 человека. Классификатор нашел троих, тогда TP=3, FN=1, FP=0, TN=0
   
   *Пример:* На фотографии 4 человека. Классификатор говорит, что людей 6 (принял куст и светофор за двух людей). Тогда TP=4, FP=2, FN=0, TN=0

In [5]:
# предполагая отсутствие false_positive:
TRUE = [4, 4, 5, 6, 5, 6, 3, 4, 5, 7]  # людей на фотографии
PRED = [3, 4, 3, 6, 5, 5, 3, 3, 4, 6]  # рамок 'person' на фотографии

true_positive = sum(PRED)
false_negative = sum(TRUE) - sum(PRED)
recall = sum(PRED) / sum(TRUE)
precision = 1
f1 = (2 * precision * recall) / (precision + recall)
print(f'{true_positive=} {false_negative=} {f1=:.3f}')

true_positive=42 false_negative=7 f1=0.923
